## RuGPT experiments

> sberbank-ai/rugpt3large_based_on_gpt2

1. Import and **unzip** texts.zip

In [ ]:
%pip install transformers
%pip install accelerate==0.27.2

## Train on merged texts

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
DEVICE

device(type='cuda')

In [2]:
from transformers import TextDataset, DataCollatorForLanguageModeling
import os

# Путь к папке с текстовыми файлами
folder_path = '/home/jupyter/datasphere/project/texts/'

# Создадим единый файл с обучающим текстом, содержащий все файлы из папки
train_path = 'train_dataset.txt'
with open(train_path, "w", encoding="utf-8") as train_file:
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, "r", encoding="utf-8") as file:
                train_file.write(file.read() + '\n')  # Разделяем тексты пустой строкой

with open(train_path, 'r') as f:
    lines = f.readlines()
# Filter out empty lines
non_empty_lines = [line for line in lines if line.strip()]

train_path = 'train_dataset_stripped.txt'
with open(train_path, 'w', encoding="utf-8") as f:
    f.writelines(non_empty_lines)


def save_first_n_lines(input_file, output_file, n):
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()[:n]

    with open(output_file, 'w', encoding='utf-8') as f:
        f.writelines(lines)

input_file = train_path
output_file = 'train_dataset_stripped1.txt'
n = 480  # Здесь указывается количество строк, которые вы хотите оставить

save_first_n_lines(input_file, output_file, n)
train_path = output_file

    
    
# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=64)
  
# Создание даталодера (нарезает текст на оптимальные по длине куски)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, 
                                                mlm=False)


2024-04-09 17:55:53.531394: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/jupyter/.local/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [3]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned", # The output directory
    overwrite_output_dir=True, # Overwrite the content of the output dir
    num_train_epochs=200, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    warmup_steps=10, # number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=16, # to make "virtual" batch size larger
    fp16=True,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5), None)
)

In [4]:
trainer.train()

100%|██████████| 200/200 [03:41<00:00,  1.11s/it]

{'train_runtime': 221.3393, 'train_samples_per_second': 65.058, 'train_steps_per_second': 0.904, 'train_loss': 0.2417168426513672, 'epoch': 178.0}


TrainOutput(global_step=200, training_loss=0.2417168426513672, metrics={'train_runtime': 221.3393, 'train_samples_per_second': 65.058, 'train_steps_per_second': 0.904, 'train_loss': 0.2417168426513672, 'epoch': 178.0})

In [5]:
import pickle
file_path = "model_rugpt3large.pkl"
with open(file_path, 'wb') as f:
    pickle.dump(model, f)

In [6]:
#############
text = "Зафайнтюнил я модель"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)


model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                        do_sample=True,
                        num_beams=2,
                        temperature=3.5,
                        top_p=0.9,
                        max_length=128,
                        )

generated_text = list(map(tokenizer.decode, out))[0]

print(generated_text)

Зафайнтюнил я модель,
Закачал я драйвера,
Установил я винду
Куча мала плюсов...
Но для тебя... увы
Нет в арсенале меня
А если бы был бы вдруг я,
То ты бы всё равно комп
Так что нет меня с тобой
Ну что же мне делать тебе
Да вот же он я за стеной
Ты б хоть спросила кто я
Да ладно что уж тут спорить
Я Зафайнтюнил модель
И поставил её на но....<s>
У каждого есть свой особый взгляд 
И свои секреты в нём есть....



In [6]:
text = "Зафайнтюнил я модель"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)


model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                        do_sample=True,
                        num_beams=2,
                        temperature=3.5,
                        top_p=0.9,
                        max_length=128,
                        )

generated_text = list(map(tokenizer.decode, out))[0]

print(generated_text)

Зафайнтюнил я модель,
Закачал я драйвера,
Установил я винду
Куча мала плюсов...
Но для тебя... увы
Нет в арсенале меня
А если бы был бы вдруг я,
То ты бы всё равно комп
Так что нет меня с тобой
Ну что же мне делать тебе
Да вот же он я за стеной
Ты б хоть спросила кто я
Да ладно что уж тут спорить
Я Зафайнтюнил модель
И поставил её на но....<s>
У каждого есть свой особый взгляд 
И свои секреты в нём есть....



## Add training on each poem

In [7]:
file_paths = []

# Путь к папке с текстовыми файлами
folder_path = '/home/jupyter/datasphere/project/texts/'

# Создадим единый файл с обучающим текстом, содержащий все файлы из папки
train_path = 'train_dataset.txt'
with open(train_path, "w", encoding="utf-8") as train_file:
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt') and filename.startswith('A') :
            file_path = os.path.join(folder_path, filename)
            
            file_paths.append(file_path)
                
print(file_paths[-1])


train_dataset = [TextDataset(tokenizer=tokenizer,file_path=file_paths[i],block_size=64) for i in range(len(file_paths))]
  
# Создание даталодера (нарезает текст на оптимальные по длине куски)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, 
                                                mlm=False)

/home/jupyter/datasphere/project/texts/A9.txt


/home/jupyter/.local/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned", # The output directory
    overwrite_output_dir=True, # Overwrite the content of the output dir
    num_train_epochs=200, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    warmup_steps=10, # number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=16, # to make "virtual" batch size larger
    fp16=True,
    )

for i in range(5): #len(file_paths)): # ЛУЧШЕ ПОСТАВИТЬ МЕНЬШЕ ИТЕРАЦИЙ
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset[i],
        optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5), None)
    )
    trainer.train()

100%|██████████| 200/200 [00:44<00:00,  4.50it/s]


{'train_runtime': 44.4612, 'train_samples_per_second': 4.498, 'train_steps_per_second': 4.498, 'train_loss': 0.001674874871969223, 'epoch': 200.0}


100%|██████████| 200/200 [01:00<00:00,  3.30it/s]


{'train_runtime': 60.5495, 'train_samples_per_second': 6.606, 'train_steps_per_second': 3.303, 'train_loss': 0.00654977798461914, 'epoch': 200.0}


100%|██████████| 200/200 [01:40<00:00,  1.99it/s]


{'train_runtime': 100.271, 'train_samples_per_second': 15.957, 'train_steps_per_second': 1.995, 'train_loss': 0.005510185360908509, 'epoch': 200.0}


100%|██████████| 200/200 [01:39<00:00,  2.02it/s]


{'train_runtime': 99.1448, 'train_samples_per_second': 10.086, 'train_steps_per_second': 2.017, 'train_loss': 0.008187416791915893, 'epoch': 200.0}


100%|██████████| 200/200 [02:13<00:00,  1.50it/s]

{'train_runtime': 133.5168, 'train_samples_per_second': 17.975, 'train_steps_per_second': 1.498, 'train_loss': 0.013275017738342285, 'epoch': 200.0}


In [10]:
import pickle
file_path = "model_rugpt3large_simple_full_text.pkl"
with open(file_path, 'wb') as f:
    pickle.dump(model, f)

In [9]:
text = "Зафайнтюнил я модель"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)


model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                        do_sample=True,
                        num_beams=2,
                        temperature=3.5,
                        top_p=0.9,
                        max_length=128,
                        )

generated_text = list(map(tokenizer.decode, out))[0]

print(generated_text)

Зафайнтюнил я модельку на минутку…

Что ж: довольно, довольно!
Ты пировать пришел, Осип!
Итак, ты доволен?
Позволь обнять себя, мой милый.
Целуй меня, мой милый, целуй.

А когда уставится тебе в очи
Свой великолепный, умный взор,
Полюби безумный, веселый нрав
И слабость жалкую души моей.

Тогда, весь дрожа, припадая к ногам
И страстно желая любви живой,
Отказываться будешь мысленно
От сладкой жизни с кукольной душой.

Но скромной


## Training on each poem

In [11]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned", # The output directory
    overwrite_output_dir=True, # Overwrite the content of the output dir
    num_train_epochs=200, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    warmup_steps=10, # number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=16, # to make "virtual" batch size larger
    fp16=True,
    )

for i in range(5): #len(file_paths)): # ЛУЧШЕ ПОСТАВИТЬ МЕНЬШЕ ИТЕРАЦИЙ
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset[i],
        optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5), None)
    )
    trainer.train()

100%|██████████| 200/200 [00:46<00:00,  4.33it/s]


{'train_runtime': 46.1863, 'train_samples_per_second': 4.33, 'train_steps_per_second': 4.33, 'train_loss': 0.018622456789016723, 'epoch': 200.0}


100%|██████████| 200/200 [01:05<00:00,  3.06it/s]


{'train_runtime': 65.3796, 'train_samples_per_second': 6.118, 'train_steps_per_second': 3.059, 'train_loss': 0.021470596790313722, 'epoch': 200.0}


100%|██████████| 200/200 [01:44<00:00,  1.91it/s]


{'train_runtime': 104.633, 'train_samples_per_second': 15.292, 'train_steps_per_second': 1.911, 'train_loss': 0.04938124179840088, 'epoch': 200.0}


100%|██████████| 200/200 [01:38<00:00,  2.03it/s]


{'train_runtime': 98.5538, 'train_samples_per_second': 10.147, 'train_steps_per_second': 2.029, 'train_loss': 0.03502564668655395, 'epoch': 200.0}


 73%|███████▎  | 146/200 [01:36<00:38,  1.41it/s]

In [ ]:
import pickle
file_path = "model_rugpt3large_full_text.pkl"
with open(file_path, 'wb') as f:
    pickle.dump(model, f)

In [ ]:
text = "Зафайнтюнил я модель"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)


model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                        do_sample=True,
                        num_beams=2,
                        temperature=3.5,
                        top_p=0.9,
                        max_length=128,
                        )

generated_text = list(map(tokenizer.decode, out))[0]

print(generated_text)